In [ ]:
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.FloatTensor)
dm.Utilities.set_compute_backend('torch')

torch.manual_seed(5)

In [ ]:
dim = 2
nb_pts = 4

gd_pos = 0.2*torch.randn(nb_pts, dim)
gd_dir = 0.2*torch.randn(nb_pts, dim)

# gd_pos = torch.tensor([[0., 1.],
#                        [1., 0]])
# gd_dir = torch.tensor([[0., 1.],
#                        [1., 0.]])

# mom_pos = torch.tensor([[0., 1.],
#                         [1., 0.]])
# mom_dir = 10.*torch.tensor([[1., 1.],
#                         [1., 1.]])

# Ici, mettre les moments à 0 (à l'un ou à l'autre) règle ce problème d'instabilité.
mom_pos = 0.2*torch.randn(nb_pts, dim)
mom_dir = 0.2*torch.randn(nb_pts, dim)

print(mom_pos)
print(mom_dir)

sigma = 0.1


oriented = dm.DeformationModules.OrientedTranslations(dim, nb_pts, sigma, 'vector', gd=(gd_pos.clone().requires_grad_(), gd_dir.clone().requires_grad_()), cotan=(mom_pos.clone().requires_grad_(), mom_dir.clone().requires_grad_()))

In [ ]:
# Nombre d'itération élevé
intermediate_states, intermediate_controls = dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([oriented]), 100, 'euler', intermediates=True)

In [ ]:
trajectories = [torch.stack(trajectory) for trajectory in list(zip(*(state[0].gd[0] for state in intermediate_states)))]

trajectories_dir = [torch.stack(trajectory_dir) for trajectory_dir in list(zip(*(state[0].gd[1] for state in intermediate_states)))]

trajectories_controls = torch.stack(tuple(control[0] for control in intermediate_controls))
oriented.compute_geodesic_control(oriented.manifold)
trajectories_controls = torch.cat([trajectories_controls, oriented.controls.detach().unsqueeze(0)]).T

trajectories_dir_controls = [control.repeat(2, 1).T*trajectory_dir for control, trajectory_dir in zip(trajectories_controls, trajectories_dir)]

In [ ]:
aabb = dm.Utilities.AABB.build_from_points(torch.cat([trajectory for trajectory in trajectories])).scale(2.)
aabb = dm.Utilities.AABB(-2., 2., -2., 2.)
print("{width}:{height}".format(width=aabb.width, height=aabb.height))

oriented = dm.DeformationModules.OrientedTranslations(dim, nb_pts, sigma, 'vector', gd=(gd_pos.clone().requires_grad_(), gd_dir.clone().requires_grad_()), cotan=(mom_pos.clone().requires_grad_(), mom_dir.clone().requires_grad_()))
width = 32
height = 32
gd_grid = aabb.fill([width, height])
nb_pts_silent = gd_grid.shape[0]
grid = dm.DeformationModules.SilentLandmarks(dim, nb_pts_silent, gd=gd_grid.clone().requires_grad_(), cotan=torch.zeros_like(gd_grid, requires_grad=True))

In [ ]:
# Pour chaque itération, print le log du conditionnement de la matrice Z lors du calcule du controle géodesique
dm.HamiltonianDynamic.shoot(dm.HamiltonianDynamic.Hamiltonian([oriented, grid]), 100, 'euler')

In [ ]:
# Controles
for c in intermediate_controls:
    print(c)

In [ ]:
%matplotlib qt5

ax = plt.subplot()

for trajectory, trajectory_dir in zip(trajectories, trajectories_dir_controls):
    plt.plot(trajectory[:, 0], trajectory[:, 1], '-')
    #plt.quiver(trajectory[:, 0], trajectory[:, 1], trajectory_dir[:, 0], trajectory_dir[:, 1], scale=200.)

#defgrid_x, defgrid_y = dm.Utilities.vec2grid(grid.manifold.gd.detach(), width, height)
#dm.Utilities.plot_grid(ax, defgrid_x, defgrid_y, color='blue')

plt.plot(gd_pos[:, 0].numpy(), gd_pos[:, 1].numpy(), 'o')

plt.axis('equal')
plt.show()

In [ ]:
%matplotlib qt5
aabb = dm.Utilities.AABB(-2., 2., -2., 2.)
print("{width}:{height}".format(width=aabb.width, height=aabb.height))

width = 32
height = 32
gd_grid = aabb.fill([width, height])

vector_field = oriented(gd_grid).detach()
oriented.compute_geodesic_control(oriented.manifold)

plt.plot(gd_pos[:, 0].numpy(), gd_pos[:, 1].numpy(), 'o')
plt.quiver(gd_grid.detach().numpy()[:, 0], gd_grid.detach().numpy()[:, 1], vector_field.numpy()[:, 0], vector_field.numpy()[:, 1], scale=20.)
plt.axis('equal')
plt.show()

In [ ]:
#print(oriented(gd_pos))
print(oriented.manifold.infinitesimal_action(oriented.field_generator()).tan)


In [ ]:
gd_pos_1 = gd_pos[0].unsqueeze(0)
gd_pos_2 = gd_pos[1].unsqueeze(0)

gd_dir_1 = gd_dir[0].unsqueeze(0)
gd_dir_2 = gd_dir[1].unsqueeze(0)

mom_pos_1 = mom_pos[0].unsqueeze(0)
mom_pos_2 = mom_pos[1].unsqueeze(0)

mom_dir_1 = mom_dir[0].unsqueeze(0)
mom_dir_2 = mom_dir[1].unsqueeze(0)


print(gd_pos_1)
print(gd_pos_2)
print(gd_dir_1)
print(gd_dir_2)
print("=====")
print(mom_pos_1)
print(mom_pos_2)
print(mom_dir_1)
print(mom_dir_2)

In [ ]:
oriented1 = dm.DeformationModules.OrientedTranslations(2, 1, sigma, gd=(gd_pos_1, gd_dir_1), cotan=(mom_pos_1, mom_dir_1))
oriented2 = dm.DeformationModules.OrientedTranslations(2, 1, sigma, gd=(gd_pos_2, gd_dir_2), cotan=(mom_pos_2, mom_dir_2))

In [ ]:
oriented1.compute_geodesic_control(oriented1.manifold)
oriented2.compute_geodesic_control(oriented2.manifold)
print(oriented1.controls)
print(oriented2.controls)

In [ ]:
field1 = oriented1.field_generator()
field2 = oriented2.field_generator()

In [ ]:
print(oriented1.manifold.infinitesimal_action(field2).tan)
print(oriented2.manifold.infinitesimal_action(field1).tan)

In [ ]:
print(dm.Kernels.rel_differences(gd_pos_1, gd_pos_2))

print(dm.Kernels.gauss_kernel(dm.Kernels.rel_differences(gd_pos_1, gd_pos_2), 1, sigma))

In [ ]:
eps = 1e-4
gd2_eps = gd_pos_2+eps*gd_dir_2
print((field1(gd2_eps) - field1(gd_pos_2))/eps)